In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta

In [145]:
df = pd.read_csv('/content/Train_details_22122017.csv')

<ipython-input-145-fbfe31628cb4>:1: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/Train_details_22122017.csv')


Given that this is a prototype and lacks actual data, I am adapting the original dataset to meet our specific requirements


Dataset link- https://github.com/itzmeanjan/indian-railway/blob/master/data/Train_details_22122017.csv

Here is the information regarding the columns present in our dataset.
Dataset information -
1. Train No .
2. Train Name
3. SEQ denotes the unique record number assigned to a specific train operating along a designated route
4. Station Code -
5. Station Name
6. Arrival time - Arrival time at a given station
7. Departure Time - departure time for the same station
8. Distance - Distance travelled by train from its original source
9. Source Station - station name in code
10. Source Station Name - Station name
11. Destination Station (code)
12. Destination Station Name

In [146]:
df.head()

,Train No,Train Name,SEQ,Station Code,Station Name,Arrival time,Departure Time,Distance,Source Station,Source Station Name,Destination Station,Destination Station Name
0,107,SWV-MAO-VLNK,1,SWV,SAWANTWADI R,00:00:00,10:25:00,0,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
1,107,SWV-MAO-VLNK,2,THVM,THIVIM,11:06:00,11:08:00,32,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
2,107,SWV-MAO-VLNK,3,KRMI,KARMALI,11:28:00,11:30:00,49,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
3,107,SWV-MAO-VLNK,4,MAO,MADGOAN JN.,12:10:00,00:00:00,78,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
4,108,VLNK-MAO-SWV,1,MAO,MADGOAN JN.,00:00:00,20:30:00,0,MAO,MADGOAN JN.,SWV,SAWANTWADI ROAD


In [147]:
# Dropping Not required columns 'Source' and 'Source Station Name'
df.drop(['Source Station','Source Station Name'], axis = 1, inplace = True)

In [148]:
df_1 = df.copy()

In [149]:
df_1.head(5)

,Train No,Train Name,SEQ,Station Code,Station Name,Arrival time,Departure Time,Distance,Destination Station,Destination Station Name
0,107,SWV-MAO-VLNK,1,SWV,SAWANTWADI R,00:00:00,10:25:00,0,MAO,MADGOAN JN.
1,107,SWV-MAO-VLNK,2,THVM,THIVIM,11:06:00,11:08:00,32,MAO,MADGOAN JN.
2,107,SWV-MAO-VLNK,3,KRMI,KARMALI,11:28:00,11:30:00,49,MAO,MADGOAN JN.
3,107,SWV-MAO-VLNK,4,MAO,MADGOAN JN.,12:10:00,00:00:00,78,MAO,MADGOAN JN.
4,108,VLNK-MAO-SWV,1,MAO,MADGOAN JN.,00:00:00,20:30:00,0,SWV,SAWANTWADI ROAD


Conversion into datetime

In [150]:
df_1['Arrival T'] = pd.to_datetime(df_1['Arrival time'],format = '%H:%M:%S',errors= 'coerce').dt.time
df_1['Departure T'] = pd.to_datetime(df_1['Departure Time'], format= '%H:%M:%S',errors= 'coerce').dt.time

Dropping more features

In [151]:
df_1.drop(['Arrival time',	'Departure Time'], axis = 1,inplace = True)

NaN/NaT values

In [155]:
df_1.isna().sum()

Train No                     0
Train Name                   0
SEQ                          0
Station Code                 0
Station Name                 0
Distance                     5
Destination Station         10
Destination Station Name    10
Arrival T                   10
Departure T                 10
dtype: int64

In [156]:
df_1.dropna(inplace=True)

In [157]:
df_1.isna().sum()

Train No                    0
Train Name                  0
SEQ                         0
Station Code                0
Station Name                0
Distance                    0
Destination Station         0
Destination Station Name    0
Arrival T                   0
Departure T                 0
dtype: int64

Grouping the data based on Train No

In [158]:
import pandas as pd
from datetime import datetime, timedelta

# Assuming df_1 is your DataFrame
grouped_df = df_1.groupby('Train No')

def calculate_duration(group):
    first_row = group[group['SEQ'] == group['SEQ'].min()]
    last_row = group[group['SEQ'] == group['SEQ'].max()]

    first_departure = first_row['Departure T'].iloc[0]
    last_arrival = last_row['Arrival T'].iloc[0]

    # Convert times to datetime objects
    first_departure = datetime.combine(datetime.min, first_departure)
    last_arrival = datetime.combine(datetime.min, last_arrival)

    if last_arrival <= first_departure:
        last_arrival += timedelta(hours=24)

    duration = last_arrival - first_departure

    cumulative_distance = group['Distance'].cumsum().iloc[-1]

    result_df = pd.DataFrame({
        'Train No': [group['Train No'].iloc[0]],
        'Train Name': [group['Train Name'].iloc[0]],
        'Duration': [duration],
        'Total Distance' : [cumulative_distance]

    })

    return result_df



In [159]:
# Apply the function to each group and concatenate the results
filtered_df = pd.concat([calculate_duration(group) for _, group in grouped_df])

# Index resetting
filtered_df.reset_index(drop=True, inplace=True)


In [162]:
filtered_df.head()

,Train No,Train Name,Duration,Total Distance
0,22989,BDTS MHV WEE,0 days 05:16:00,6113
1,22990,MHV BDTS SF,0 days 15:45:00,5732
2,22991,BDTS VRL SF,0 days 15:50:00,5848
3,22992,VRL BDTS SF,0 days 16:30:00,5071
4,22993,BDTS MHV SF,0 days 15:45:00,7573


Since we do not have the actual data, So for experiment we are adding new column in teh dataset whihc is our main target features, before that let's add one more independent feature which is 'Night travelling'

In [164]:
df = filtered_df.copy()

In [170]:
# Convert the 'Duration' column to a timedelta object
df['Duration'] = pd.to_timedelta(df['Duration'])

threshold = 18
# Apply a lambda function to create the 'Night travel' column
df['Night travel'] = df['Duration'].apply(lambda x: 1 if x.total_seconds() > threshold * 3600 else 0)


In [171]:
df.head(5)

,Train No,Train Name,Duration,Total Distance,Night travel
0,22989,BDTS MHV WEE,0 days 05:16:00,6113,0
1,22990,MHV BDTS SF,0 days 15:45:00,5732,0
2,22991,BDTS VRL SF,0 days 15:50:00,5848,0
3,22992,VRL BDTS SF,0 days 16:30:00,5071,0
4,22993,BDTS MHV SF,0 days 15:45:00,7573,0


In [172]:
# Check if we have some rows where Travelling duration exceeds 18 hours
df[df['Night travel'] == 1].head()

,Train No,Train Name,Duration,Total Distance,Night travel
20,24369,TRIVENI EXPR,0 days 20:40:00,11205,1
21,24370,TRIVENI EXP,0 days 20:00:00,16914,1
2632,51188,KTE BSL PASS,0 days 18:40:00,23598,1
2633,51189,ET ALD PASS,0 days 22:10:00,20159,1
2634,51190,ALD ET PASS,0 days 20:10:00,18817,1


Adding dummy dependent feature - '*Willing to share*'

In [173]:
df['Willing to share'] = df['Night travel'].apply(lambda x :'yes' if x == 1 else 'No')

In [174]:
df.head()

,Train No,Train Name,Duration,Total Distance,Night travel,Willing to share
0,22989,BDTS MHV WEE,0 days 05:16:00,6113,0,No
1,22990,MHV BDTS SF,0 days 15:45:00,5732,0,No
2,22991,BDTS VRL SF,0 days 15:50:00,5848,0,No
3,22992,VRL BDTS SF,0 days 16:30:00,5071,0,No
4,22993,BDTS MHV SF,0 days 15:45:00,7573,0,No


In [176]:
# Checking dataset where passenger shows willingness to share
df[df['Willing to share'] == 'yes'].head(5)

,Train No,Train Name,Duration,Total Distance,Night travel,Willing to share
20,24369,TRIVENI EXPR,0 days 20:40:00,11205,1,yes
21,24370,TRIVENI EXP,0 days 20:00:00,16914,1,yes
2632,51188,KTE BSL PASS,0 days 18:40:00,23598,1,yes
2633,51189,ET ALD PASS,0 days 22:10:00,20159,1,yes
2634,51190,ALD ET PASS,0 days 20:10:00,18817,1,yes


*Note*-Finally , You can apply any favourable machine learaning technique, which suites to this model, since here we have forcefully made the dataset with the conditino which can be unsuitable for the success of our business model, that's why further machine learning algorithm implementation will not give us fruitfull result.


**Thank you for your time**